In [ ]:
import sagemaker
import boto3

session = sagemaker.Session()
role = sagemaker.get_execution_role()

bucket = "sagemaker-churn-mlops-ka"
prefix = "sagemaker-churn-mlops"

TRAIN_PATH = f"s3://{bucket}/{prefix}/processed/train/train.csv"

print("Training data:", TRAIN_PATH)


In [ ]:
image_uri = sagemaker.image_uris.retrieve(
    framework="xgboost",
    region=session.boto_region_name,
    version="1.5-1",
    py_version="py3"
)

print("Using container:", image_uri)

In [ ]:
from sagemaker.estimator import Estimator

estimator = Estimator(
    image_uri=image_uri,
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",

    entry_point="training.py",     # <--- your script
    source_dir="training",      # <--- folder containing train.py

    output_path=f"s3://{bucket}/{prefix}/model",
    base_job_name="churn-xgb-train",
    sagemaker_session=session
)

print("Estimator created.")


In [ ]:
estimator.fit({"train": TRAIN_PATH})

In [ ]:
estimator = Estimator.attach("churn-xgb-train-2025-12-02-01-41-14-611")  # e.g., churn-training-2024-06-13-10-40-22
model_path = estimator.model_data
print("✅ Model artifact path:", model_path)